## Classes and Interfaces
### Item 37: Compose Classes Instead of Nesting Many Levels of Built-in Types
- Avoid making dictionaries with values that are dictionaries, long tuples, or complex nesting of other built-in types.
- Use `namedtuple` for lightweight, immutable data containers before you need the flexibility of a full class.
- Move your bookkeeping code to using multiple classes when your internal state dictionaries get complicated.